In [2]:
import pandas as pd
import ast
import numpy as np
import json

In [3]:
##Merging TMDB and IMDB datasets (till Jan 1,2020) 


#Import both datasets
movies=pd.read_csv('IMDB movies.csv')
new_movies=pd.read_csv('new_movie.csv')

C:\Users\Harman\.conda\envs\PDSH\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#Drop irrelevant columns

movies.drop(columns=['usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics','duration','language','date_published','budget','title','writer','votes','production_company'],inplace=True)

new_movies.drop(columns=['original_title','adult','overview'],inplace=True)

#Remove nulls
movies.dropna(inplace=True)

#Rename columns 
movies.rename(columns={'imdb_title_id': 'imdb_id'}, inplace=True)

#Merge based on imdb_id
final=pd.merge(new_movies,movies,on='imdb_id',how='inner')

#Keep only unique entries
final=final.drop_duplicates(subset='imdb_id', keep="first")

#Save to disk
final.to_csv('movie_data.csv',index=False)

(83549, 9)
(77663, 5)


In [7]:
##Preprocessing TMDB data (after Jan 1, 2020)

In [ ]:
df=pd.read_csv('tmdb_2020.csv')

In [ ]:
#Process column names so that to match with original data

In [ ]:
#Get genres
def read_genres(inp):
        df=pd.json_normalize(ast.literal_eval(inp))
        if 'name' not in df:
            return np.nan
        else:
            df.dropna(inplace=True)
            temp=df.name.tolist()
            return json.dumps(temp)

In [ ]:
#